In [1]:
%matplotlib inline

In [2]:
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels
from maskfunctions import create_mask
from scipy.stats import lognorm
from scipy.stats import gamma

In [3]:
# Location and names of data files
data_dir = "/Users/jeremysmith/Documents/BTS_Airline_Data/"
file_pre = "On_Time_On_Time_Performance_2016"
path_airports = os.path.join(data_dir, "airports.dat")
path_emplanes = os.path.join(data_dir, "cy15-commercial-service-enplanements.csv")

In [4]:
# Delay Data
data_list = []
usecols = [2,3,4,5,8,10,14,23,31,33,36,39,42,44,47,52,54,56,57,58,59,60]
for month in range(12):
    file_month = "{:s}_{:02d}".format(file_pre, month + 1)
    path_month = os.path.join(data_dir, file_month, file_month + ".csv")
    data_list.append(pd.read_csv(path_month, usecols=usecols))
data_delays = pd.concat(data_list, ignore_index=True)

In [5]:
data_delays.head()

,Month,DayofMonth,DayOfWeek,FlightDate,Carrier,FlightNum,Origin,Dest,DepDelay,DepDel15,...,ArrDelay,ArrDel15,Cancelled,AirTime,Distance,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
0,1,6,3,2016-01-06,AA,43,DFW,DTW,-3.0,0.0,...,-6.0,0.0,0.0,132.0,986.0,NaN,NaN,NaN,NaN,NaN
1,1,7,4,2016-01-07,AA,43,DFW,DTW,-4.0,0.0,...,-12.0,0.0,0.0,126.0,986.0,NaN,NaN,NaN,NaN,NaN
2,1,8,5,2016-01-08,AA,43,DFW,DTW,-5.0,0.0,...,7.0,0.0,0.0,135.0,986.0,NaN,NaN,NaN,NaN,NaN
3,1,9,6,2016-01-09,AA,43,DFW,DTW,2.0,0.0,...,-5.0,0.0,0.0,129.0,986.0,NaN,NaN,NaN,NaN,NaN
4,1,10,7,2016-01-10,AA,43,DFW,DTW,100.0,1.0,...,113.0,1.0,0.0,137.0,986.0,0.0,0.0,47.0,0.0,66.0


In [6]:
# Airport Location Data
head_airports = ['Name', 'City', 'Country', 'IATA',
                 'Latitude', 'Longitude', 'Altitude',
                 'Timezone']
data_airports = pd.read_csv(path_airports,
                           usecols=[1,2,3,4,6,7,8,9],
                           names=head_airports)

In [7]:
# Airport USA Enplanements Data
data_emplanes = pd.read_csv(path_emplanes,
                           usecols=[2,3,6,7,8],
                           thousands=',')
data_emplanes.rename(columns={'Locid':'IATA', 'CY 15 Enplanements':'CY15enplane'}, inplace=True)

In [8]:
# Merge Airport Data
data_airports_merged = pd.merge(data_emplanes, data_airports, on='IATA', how='left', copy=False)

In [9]:
data_airports_merged_us = data_airports_merged[data_airports_merged.Country == "United States"]
data_airports_merged_us = data_airports_merged_us[data_airports_merged_us.CY15enplane > 100000]
data_airports_merged_us.head()

,ST,IATA,S/L,Hub,CY15enplane,Name,City,Country,Latitude,Longitude,Altitude,Timezone
0,GA,ATL,P,L,49340732,Hartsfield Jackson Atlanta International Airport,Atlanta,United States,33.636700,-84.428101,1026.0,-5
1,CA,LAX,P,L,36351272,Los Angeles International Airport,Los Angeles,United States,33.942501,-118.407997,125.0,-8
2,IL,ORD,P,L,36305668,Chicago O'Hare International Airport,Chicago,United States,41.978600,-87.904800,672.0,-6
3,TX,DFW,P,L,31589839,Dallas Fort Worth International Airport,Dallas-Fort Worth,United States,32.896801,-97.038002,607.0,-6
4,NY,JFK,P,L,27782369,John F Kennedy International Airport,New York,United States,40.639801,-73.778900,13.0,-5


In [10]:
# Output Airport Data Without Delay Data Embedded
data_airports_merged_us.to_csv("visualization/airports.csv")

In [40]:
# Setup for filter and mask
airports = data_airports_merged_us['IATA'].values
carrier = ["ALL", "AA", "DL", "WN", "UA", "AS", "B6", "NK"]
month = [0, 'SPR', 'SUM', 'FAL', 'WIN']
dotw = 0

In [43]:
# Loop to generate files
for c in carrier:
    for m in month:
        # Create Filter and Mask
        mask, matches = create_mask(data_delays, airports, c, m, dotw)
        print m, c
        print " There are {:d} matching flights for this filter".format(matches)

        # Information grouped by destination airport
        groupDest = data_delays[mask].groupby('Dest', as_index=False)

        num_arr = groupDest.size().to_frame().reset_index().rename(columns={0:'ArrNum'})
        fraction_delayed = groupDest['ArrDel15'].mean()
        med_in_time = groupDest['TaxiIn','ArrDelay'].median()
        mergeDest = pd.merge(fraction_delayed, med_in_time, on="Dest").merge(num_arr, on="Dest")
        mergeDest.ArrDel15 *= 100
        mergeDest.rename(columns={'Dest':'IATA', 'ArrDel15':'PercentArrDel15'}, inplace=True)

        # Information grouped by origin airport
        groupOrig = data_delays[mask].groupby('Origin', as_index=False)

        num_dep = groupOrig.size().to_frame().reset_index().rename(columns={0:'DepNum'})
        fraction_cancel = groupOrig['Cancelled'].mean()
        med_out_time = groupOrig['TaxiOut', 'DepDelay'].median()
        mergeOrig = pd.merge(fraction_cancel, med_out_time, on="Origin").merge(num_dep, on="Origin")
        mergeOrig.Cancelled *= 100
        mergeOrig.rename(columns={'Origin':'IATA', 'Cancelled':'PercentCancelled'}, inplace=True)

        # Merge Origin and Destination delay data
        mergeAll = pd.merge(mergeDest, mergeOrig, on='IATA')

        # Merge delay data into the airport information df
        data_airports_merged_us_wdelay = pd.merge(data_airports_merged_us,
                                                  mergeAll, on='IATA', how='inner', copy=False)

        # Output Airport Data With Delay Data Embedded
        data_airports_merged_us_wdelay.to_csv("visualization/airports_wdelaydata_{:s}_{:s}.csv".format(m if m != 0 else 'ALL', c))

0 ALL
 There are 5508157 matching flights for this filter
SPR ALL
 There are 1392506 matching flights for this filter
SUM ALL
 There are 1458877 matching flights for this filter
FAL ALL
 There are 1353224 matching flights for this filter
WIN ALL
 There are 1303550 matching flights for this filter
0 AA
 There are 905054 matching flights for this filter
SPR AA
 There are 230031 matching flights for this filter
SUM AA
 There are 239105 matching flights for this filter
FAL AA
 There are 217686 matching flights for this filter
WIN AA
 There are 218232 matching flights for this filter
0 DL
 There are 918285 matching flights for this filter
SPR DL
 There are 234169 matching flights for this filter
SUM DL
 There are 248597 matching flights for this filter
FAL DL
 There are 227698 matching flights for this filter
WIN DL
 There are 207821 matching flights for this filter
0 WN
 There are 1296078 matching flights for this filter
SPR WN
 There are 328837 matching flights for this filter
SUM WN
 The